In [1]:
# Importar as bibliotecas

# -*- coding: utf-8 -*-

import numpy as np
from timeit import default_timer as timer
import random
import serial
import serial.tools.list_ports
import os
from math import sqrt
import argparse
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from datetime import datetime
from scipy.fft import fft

# https://www.linkedin.com/pulse/lendo-arquivos-csv-com-pandas-rog%C3%A9rio-guimar%C3%A3es-de-campos-j%C3%BAnior/
# https://stackabuse.com/python-list-files-in-a-directory/

C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [4]:
# Carregar os dados 

# Definição do paciente e os diretórios
nome = "Anderson"
movimentos = ["repouso", "flexao_polegar", "flexao_indicador", "flexao_total", "extensao_total"]

# Definir DataFrame
df = {}

# Declarando quais movimentos foram realizados
tipo_de_movimento = []
relacao_de_movimentos = {
    'repouso':0,
    'flexao_total':1,
    'extensao_total':2,
    'flexao_polegar':3,
    'flexao_indicador':4,
}


for movimento in movimentos:
    #Caminho dos dados
    filename = f"Dataset\\{nome}\\{movimento}\\dados_{movimento}.csv"

    # Definir lables dos dados
    if movimento == "repouso":
        names = ['Full']
    else:
        names = ['Full', 'Part', 'RMS', 'FFT', 'FREQ_FFT']
    
    # Colocar cada Dataframe dentro de um dicionário
    df[f'{movimento}'] = pd.read_csv(f"{filename}", sep=";", header=None, names=names, index_col=None)
    
    # Extrair quantidade de linhas para cada movimento
    height = df[f'{movimento}'].shape()[0]
    
    # Preencher um vetor com o tipo de movimento
    for row in range(0, height):
        tipo_de_movimento.append(relacao_de_movimentos[f'{movimento}'])
    
    df[f'{movimento}']['Tipo_de_Movimento'] = tipo_de_movimento

df['extensao_total'].info()
display(df['extensao_total'].describe())
display(df['extensao_total'].columns)
display(df['extensao_total'].shape)

# ADICIONAR UMA COLUNA INDICANDO QUAL É O MOVIMENTO REALIZADO DE 0 A 4 DEPENDENDO DE CADA MOVIMENTO

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Full      19 non-null     object
 1   Part      19 non-null     object
 2   RMS       19 non-null     object
 3   FFT       2 non-null      object
 4   FREQ_FFT  1 non-null      object
dtypes: object(5)
memory usage: 888.0+ bytes


,Full,Part,RMS,FFT,FREQ_FFT
count,19,19,19,2,1
unique,19,19,19,2,1
top,"[0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 20....","[2.0, 10.0, 16.0, 18.0, 24.0, 28.0, 32.0, 32.0...","[38.44769173825654, 15.962455951387932, 15.916...","[(5638-0j), (826.5913373081404-245.35572270174...","[0.0, 2.0100502512562812, 4.0201005025125625, ..."
freq,1,1,1,1,1


Index(['Full', 'Part', 'RMS', 'FFT', 'FREQ_FFT'], dtype='object')

(19, 5)

In [4]:
# Carregar os dados 

# Definição do paciente e os diretórios
nome = "Fulano"
movimentos = ["repouso", "flexao_polegar", "flexao_indicador", "flexao_total", "extensao_total"]

# Definir DataFrame
df = {}

# Declarando quais movimentos foram realizados
tipo_de_movimento = []
relacao_de_movimentos = {
    'repouso':0,
    'flexao_total':1,
    'extensao_total':2,
    'flexao_polegar':3,
    'flexao_indicador':4,
}


for movimento in movimentos:
    #Caminho dos dados
    filename = f"Dataset\\{nome}\\{movimento}\\dados_{movimento}.csv"

    # Definir lables dos dados
    if movimento == "repouso":
        names = ['Full']
    else:
        names = ['Full', 'Part', 'RMS', 'FFT', 'FREQ_FFT']
    
    # Colocar cada Dataframe dentro de um dicionário
    df[f'{movimento}'] = pd.read_csv(f"{filename}", sep=";", header=None, names=names, index_col=None)
    
    # Extrair quantidade de linhas para cada movimento
    height = df[f'{movimento}'].shape()[0]
    
    # Preencher um vetor com o tipo de movimento
    for row in range(0, height):
        tipo_de_movimento.append(relacao_de_movimentos[f'{movimento}'])
    
    df[f'{movimento}']['Tipo_de_Movimento'] = tipo_de_movimento

df['extensao_total'].info()
display(df['extensao_total'].describe())
display(df['extensao_total'].columns)
display(df['extensao_total'].shape)

# ADICIONAR UMA COLUNA INDICANDO QUAL É O MOVIMENTO REALIZADO DE 0 A 4 DEPENDENDO DE CADA MOVIMENTO

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Full      19 non-null     object
 1   Part      19 non-null     object
 2   RMS       19 non-null     object
 3   FFT       2 non-null      object
 4   FREQ_FFT  1 non-null      object
dtypes: object(5)
memory usage: 888.0+ bytes


,Full,Part,RMS,FFT,FREQ_FFT
count,19,19,19,2,1
unique,19,19,19,2,1
top,"[0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 20....","[2.0, 10.0, 16.0, 18.0, 24.0, 28.0, 32.0, 32.0...","[38.44769173825654, 15.962455951387932, 15.916...","[(5638-0j), (826.5913373081404-245.35572270174...","[0.0, 2.0100502512562812, 4.0201005025125625, ..."
freq,1,1,1,1,1


Index(['Full', 'Part', 'RMS', 'FFT', 'FREQ_FFT'], dtype='object')

(19, 5)

In [5]:
relacao_de_movimentos = {
    'repouso':0,
    'felxao':1,
    'extensao':2,
    'flexao_polegar':3,
    'flexao_indicador':4,
}

print(relacao_de_movimentos['repouso'])

0


In [9]:
# Treinar e Testar
from sklearn.model_selection import train_test_split
    
# https://gusrabbit.com/intuition/treino-teste/

In [8]:
# Criar a rede LSTM

In [ ]:
# Criar a rede GRU

In [10]:
# Verificar acurácia e perda